In [0]:
from google.colab import drive 
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
cd /content/drive/'My Drive'/annd/nlp

/content/drive/My Drive/annd/nlp


In [0]:
import pandas as pd
df_train = pd.read_csv('train_txt', sep='\t')
df_valid = pd.read_csv('valid_txt', sep='\t')
df_test = pd.read_csv('test_txt', sep='\t')

In [0]:
category={'b' : 0, 't' : 1, 'e' : 2, 'm' :3 }
train1 = df_train[['Title', 'Category']]
valid1 = df_valid[['Title', 'Category']]
test1 = df_test[['Title', 'Category']]
train1['Category'] = train1['Category'].map(category)
valid1['Category'] = valid1['Category'].map(category)
test1['Category'] = test1['Category'].map(category)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

In [0]:
train1.to_csv('train1.txt', sep = '\t', index=False, header=None)
valid1.to_csv('valid1.txt', sep = '\t', index=False, header=None)
test1.to_csv('test1.txt', sep = '\t', index=False, header=None)

# 80. ID番号への変換
問題51で構築した学習データ中の単語にユニークなID番号を付与したい．学習データ中で最も頻出する単語に1，2番目に頻出する単語に2，……といった方法で，学習データ中で2回以上出現する単語にID番号を付与せよ．そして，与えられた単語列に対して，ID番号の列を返す関数を実装せよ．ただし，出現頻度が2回未満の単語のID番号はすべて0とせよ

In [0]:
from torchtext import data
import torch

In [0]:
Title = data.Field(sequential=True, lower=True, batch_first=True)
Category = data.Field(sequential=False, batch_first=True, use_vocab=False)

train, valid, test = data.TabularDataset.splits(path = '.', train='train1.txt',
    validation='valid1.txt', test='test1.txt', format='tsv',
    fields=[('Title', Title), ('Category', Category)])

In [0]:
Title.build_vocab(train, min_freq=2)
print(Title.vocab.stoi)

defaultdict(<function _default_unk_index at 0x7fa8ed34d158>, {'<unk>': 0, '<pad>': 1, 'to': 2, '...': 3, 'in': 4, 'the': 5, 'of': 6, 'for': 7, 'on': 8, 'as': 9, 'update': 10, '-': 11, 'and': 12, 'us': 13, 'a': 14, 'with': 15, 'at': 16, 'is': 17, 'after': 18, 'new': 19, 'says': 20, 'from': 21, 'by': 22, 'up': 23, 'her': 24, 'kardashian': 25, 'over': 26, 'kim': 27, 'be': 28, 'first': 29, 'more': 30, 'stocks': 31, 'about': 32, 'are': 33, 'may': 34, 'shares': 35, 'china': 36, 'not': 37, 'out': 38, 'that': 39, 'will': 40, '(1)': 41, 'euro': 42, 'it': 43, 'deal': 44, 'sales': 45, 'miley': 46, 'data': 47, 'ecb': 48, 'she': 49, 'off': 50, 'profit': 51, 'fed': 52, 'you': 53, 'global': 54, 'his': 55, 'but': 56, 'billion': 57, 'cyrus': 58, 'rise': 59, 'has': 60, 'bank': 61, 'was': 62, 'ceo': 63, 'bln': 64, 'chris': 65, 'st': 66, 'growth': 67, 'set': 68, 'could': 69, 'have': 70, 'west': 71, 'high': 72, 'million': 73, 'than': 74, 'into': 75, 'kanye': 76, 'day': 77, 'falls': 78, 'home': 79, 'low': 8

# 81. RNNによる予測
ID番号で表現された単語列x=(x1,x2,…,xT)がある．ただし，Tは単語列の長さ，xt∈ℝVは単語のID番号のone-hot表記である（Vは単語の総数である）．再帰型ニューラルネットワーク（RNN: Recurrent Neural Network）を用い，単語列xからカテゴリyを予測するモデルとして，次式を実装せよ．

h→0=0,h→t=RNN−→−−(emb(xt),h→t−1),y=softmax(W(yh)h→T+b(y))
ただし，emb(x)∈ℝdwは単語埋め込み（単語のone-hot表記から単語ベクトルに変換する関数），h→t∈ℝdhは時刻tの隠れ状態ベクトル，RNN−→−−(x,h)は入力xと前時刻の隠れ状態hから次状態を計算するRNNユニット，W(yh)∈ℝL×dhは隠れ状態ベクトルからカテゴリを予測するための行列，b(y)∈ℝLはバイアス項である（dw,dh,Lはそれぞれ，単語埋め込みの次元数，隠れ状態ベクトルの次元数，ラベル数である）．RNNユニットRNN−→−−(x,h)には様々な構成が考えられるが，典型例として次式が挙げられる．

RNN−→−−(x,h)=g(W(hx)x+W(hh)h+b(h))
ただし，W(hx)∈ℝdh×dw，W(hh)∈ℝdh×dh,b(h)∈ℝdhはRNNユニットのパラメータ，gは活性化関数（例えばtanhやReLUなど）である．

なお，この問題ではパラメータの学習を行わず，ランダムに初期化されたパラメータでyを計算するだけでよい．次元数などのハイパーパラメータは，dw=300,dh=50など，適当な値に設定せよ（以降の問題でも同様である）．

In [0]:
import torch
from gensim.models import KeyedVectors

In [0]:
class RNN(torch.nn.Module):
    def __init__(self, num_embeddings,
                 embedding_dim=300,
                 hidden_size=300,
                 output_size=1,
                 num_layers=1,
                 dropout=0.2):
        super().__init__()
        model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)
        weights = torch.FloatTensor(model.vectors)
        self.emb = torch.nn.Embedding.from_pretrained(weights)
        self.lstm = torch.nn.LSTM(embedding_dim,
                            hidden_size, num_layers,
                            batch_first=True, dropout=dropout)
        self.linear = torch.nn.Linear(hidden_size, output_size)

    def forward(self, x, h0=None):
        x = self.emb(x)
        x, h = self.lstm(x, h0)
        x = x[:, -1, :]
        x = self.linear(x)
        return x

In [0]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [0]:
import torchtext
train_iter = torchtext.data.BucketIterator(dataset=train, batch_size=32, sort_key=lambda x: len(x.Title), shuffle = True)
valid_iter = torchtext.data.BucketIterator(dataset=valid, batch_size=32, sort_key=lambda x: len(x.Title), shuffle = True)
test_iter = torchtext.data.BucketIterator(dataset=test, batch_size=32, sort_key=lambda x: len(x.Title), shuffle = True)

In [0]:
Title.build_vocab(train, min_freq=2)
Category.build_vocab(train)


In [0]:
model = RNN(len(Title.vocab.stoi) + 1, num_layers=2, output_size=4)
model.to(device)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


RNN(
  (emb): Embedding(3000000, 300)
  (lstm): LSTM(300, 300, num_layers=2, batch_first=True, dropout=0.2)
  (linear): Linear(in_features=300, out_features=4, bias=True)
)

In [0]:
opt = torch.optim.Adam(model.parameters(), lr=0.001)

In [0]:
criterion = torch.nn.CrossEntropyLoss()

In [0]:
EPOCHS=20

In [0]:

for epoch in range(EPOCHS):
    model.train()
    for i, batch in enumerate(train_iter):
        x, y = batch.Title, batch.Category
        y_pred = model(x.to(device))
        loss = criterion(y_pred.to(device), y.to(device))
        opt.zero_grad()
        loss.backward()
        opt.step()
        print('\rEpoch: %d/%d, %f%% loss: %0.2f'% (epoch+1, EPOCHS, i/len(train_iter)*100, loss.item()), end='')

Epoch: 20/20, 99.700599% loss: 0.05

In [0]:
def accuracy(y_actual, y_pred):
    return np.sum(y_actual == y_pred).astype('int') / y_actual.shape[0]

In [0]:
import numpy as np
avg_acc=0
for epoch in range(1):
    model.eval()
    for i, batch in enumerate(valid_iter):
        x, y = batch.Title, batch.Category
        y_pred = model(x.to(device))
        acc = accuracy(y.cpu().numpy(), torch.max(y_pred.data, 1)[1].cpu().detach().numpy().squeeze())
        avg_acc += acc
        print('\r%0.2f%% loss: %0.2f, accuracy %0.2f'% (i/len(valid_iter)*100, loss.item(), acc), end='')
print('\nAverage accuracy: ', avg_acc / len(valid_iter))

97.62% loss: 0.05, accuracy 0.81
Average accuracy:  0.778814935064935


In [0]:
avg_acc=0

for epoch in range(1):
    model.eval()
    for i, batch in enumerate(test_iter):
        x, y = batch.Title, batch.Category
        y_pred = model(x.to(device))
        acc = accuracy(y.cpu().numpy(), torch.max(y_pred.data, 1)[1].cpu().detach().numpy().squeeze())
        avg_acc += acc
        print('\r%0.2f%% loss: %0.2f, accuracy %0.2f'% (i/len(test_iter)*100, loss.item(), acc), end='')
print('\nAverage accuracy: ', avg_acc / len(test_iter))

97.62% loss: 0.05, accuracy 0.78
Average accuracy:  0.807224025974026


86. 畳み込みニューラルネットワーク (CNN)Permalink
ID番号で表現された単語列x=(x1,x2,…,xT)がある．ただし，Tは単語列の長さ，xt∈ℝVは単語のID番号のone-hot表記である（Vは単語の総数である）．畳み込みニューラルネットワーク（CNN: Convolutional Neural Network）を用い，単語列xからカテゴリyを予測するモデルを実装せよ．

ただし，畳み込みニューラルネットワークの構成は以下の通りとする．

単語埋め込みの次元数: dw
畳み込みのフィルターのサイズ: 3 トークン
畳み込みのストライド: 1 トークン
畳み込みのパディング: あり
畳み込み演算後の各時刻のベクトルの次元数: dh
畳み込み演算後に最大値プーリング（max pooling）を適用し，入力文をdh次元の隠れベクトルで表現
すなわち，時刻tの特徴ベクトルpt∈ℝdhは次式で表される．

pt=g(W(px)[emb(xt−1);emb(xt);emb(xt+1)]+b(p))
ただし，W(px)∈ℝdh×3dw,b(p)∈ℝdhはCNNのパラメータ，gは活性化関数（例えばtanhやReLUなど），[a;b;c]はベクトルa,b,cの連結である．なお，行列W(px)の列数が3dwになるのは，3個のトークンの単語埋め込みを連結したものに対して，線形変換を行うためである．

最大値プーリングでは，特徴ベクトルの次元毎に全時刻における最大値を取り，入力文書の特徴ベクトルc∈ℝdhを求める．c[i]でベクトルcのi番目の次元の値を表すことにすると，最大値プーリングは次式で表される．

c[i]=max1≤t≤Tpt[i]
最後に，入力文書の特徴ベクトルcに行列W(yc)∈ℝL×dhとバイアス項b(y)∈ℝLによる線形変換とソフトマックス関数を適用し，カテゴリyを予測する．

y=softmax(W(yc)c+b(y))
なお，この問題ではモデルの学習を行わず，ランダムに初期化された重み行列でyを計算するだけでよい．

In [0]:
import torch.nn.functional as F
from torch import nn, optim
class CNN(nn.Module):

    def __init__(self, output_dim, kernel_num, dropout=0.1, static=False):
        super(CNN, self).__init__()

        model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)
        weights = torch.FloatTensor(model.vectors)
        self.embed = nn.Embedding.from_pretrained(weights)
        self.conv13 = nn.Conv2d(1, kernel_num, (1, self.embed.weight.shape[1]))
        self.conv14 = nn.Conv2d(1, kernel_num, (2, self.embed.weight.shape[1]))
        self.conv15 = nn.Conv2d(1, kernel_num, (3, self.embed.weight.shape[1]))
        self.dropout = nn.Dropout(dropout)
        self.fc1 = nn.Linear(3 * kernel_num, output_dim)

    def conv_and_pool(self, x, conv):
        x = F.relu(conv(x)).squeeze(3) 
        x = F.max_pool1d(x, x.size(2)).squeeze(2)
        return x

    def forward(self, x):
        x = self.embed(x)
        x = x.unsqueeze(1)
        x = x.float()
        x1 = self.conv_and_pool(x,self.conv13)
        x2 = self.conv_and_pool(x,self.conv14)
        x3 = self.conv_and_pool(x,self.conv15)
        x = torch.cat((x1, x2, x3), 1)
        x = self.dropout(x)
        logit = self.fc1(x)
        return logit

In [0]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [0]:
model = CNN(output_dim=4, kernel_num=3, dropout=0.1)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
model.to(device)

CNN(
  (embed): Embedding(3000000, 300)
  (conv13): Conv2d(1, 3, kernel_size=(1, 300), stride=(1, 1))
  (conv14): Conv2d(1, 3, kernel_size=(2, 300), stride=(1, 1))
  (conv15): Conv2d(1, 3, kernel_size=(3, 300), stride=(1, 1))
  (dropout): Dropout(p=0.1, inplace=False)
  (fc1): Linear(in_features=9, out_features=4, bias=True)
)

In [0]:
criterion = torch.nn.CrossEntropyLoss()
opt = torch.optim.Adam(model.parameters(), lr=0.001)

In [0]:
for epoch in range(EPOCHS):
    model.train()
    for i, batch in enumerate(train_iter):
        x, y = batch.Title, batch.Category
        y_pred = model(x.to(device))
        loss = criterion(y_pred.to(device), y.to(device))
        opt.zero_grad()
        loss.backward()
        opt.step()
        print('\rEpoch: %d/%d, %f%% loss: %0.2f'% (epoch+1, EPOCHS, i/len(train_iter)*100, loss.item()), end='')

Epoch: 20/20, 99.700599% loss: 0.34

In [0]:
import numpy as np
avg_acc=0
for epoch in range(1):
    model.eval()
    for i, batch in enumerate(valid_iter):
        x, y = batch.Title, batch.Category
        y_pred = model(x.to(device))
        acc = accuracy(y.cpu().numpy(), torch.max(y_pred.data, 1)[1].cpu().detach().numpy().squeeze())
        avg_acc += acc
        print('\r%0.2f%% loss: %0.2f, accuracy %0.2f'% (i/len(valid_iter)*100, loss.item(), acc), end='')
print('\nAverage accuracy: ', avg_acc / len(valid_iter))

97.62% loss: 0.34, accuracy 0.69
Average accuracy:  0.7359983766233766


In [0]:
avg_acc=0
for epoch in range(1):
    model.eval()
    for i, batch in enumerate(test_iter):
        x, y = batch.Title, batch.Category
        y_pred = model(x.to(device))
        acc = accuracy(y.cpu().numpy(), torch.max(y_pred.data, 1)[1].cpu().detach().numpy().squeeze())
        avg_acc += acc
        print('\r%0.2f%% loss: %0.2f, accuracy %0.2f'% (i/len(test_iter)*100, loss.item(), acc), end='')
print('\nAverage accuracy: ', avg_acc / len(test_iter))

97.62% loss: 0.34, accuracy 0.84
Average accuracy:  0.7754329004329005
